In [ ]:
import os
import statistics
import scipy as sp
import math
import pandas as pd
#pd.get_option("display.max_rows")
#pd.reset_option("display.max_rows")
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, RegressorMixin
%matplotlib inline

In [ ]:
os.chdir(' ') #set proper directory depending where you put that project
#data - data for 2019/2020
#data05 - data for 2017/2018
#data1 - data for 2018/19
#data15 - data for 2020/21
data = pd.read_csv('danelic2019.csv',sep=';',engine='python')
data05 =  pd.read_csv('danelic2017.csv',sep=';',engine='python')
data1 = pd.read_csv('danelic2018.csv',sep=';',engine='python')
#data15 = pd.read_csv('danelic2020.csv',sep=';',engine='python')
data1=pd.DataFrame.append(data,data1)
data1=pd.DataFrame.append(data1,data05,ignore_index=True)
#data1=pd.DataFrame.append(data1,data15,ignore_index=True)
data1.sort_values('player')

In [ ]:
#adding dummy variables to dataset
data1 = pd.get_dummies(data1, columns=['league'])
data1 = data1.rename({"league_Bundesliga":"isBundesliga",
                                "league_La Liga":"isLaLiga",
                                "league_Premier League":"isPremierLeague",
                                "league_Ligue 1":"isLigue1",
                                "league_Serie A":"isSerieA"},axis='columns')
data1=pd.get_dummies(data1,columns=['Season'])
data1=pd.get_dummies(data1,columns=['foot'])
#deleting potential outliers that actually contribute nothing
data1=data1[data1['value']>1000000]
data1=data1[data1['games']>5]
data1=data1[data1['age']>0]
data1=data1[data1['height']>0]
data1

In [ ]:
#DEFENDERS
dataDEF = data1[data1['position2'].str[:8]=='Defender']
dataDEF

In [ ]:
#was originally used for correlations calculations (visible in appendix)

#dataDEF['age']=dataDEF['age']*dataDEF['age']
#dataDEF_cor = dataDEF.corr()
#show=pd.Series(dataDEF_cor['value'])
#pd.set_option('display.max_rows', None)
#show.sort_values(ascending=False)

In [ ]:
#a cosmetic change in log notation for my liking
def ln(x):
    return np.log(x)

dataDEF['pctpassesshort']=(dataDEF['passes_short']/dataDEF['passes'])/dataDEF['minutes']
dataDEF['pctpassesmedium']=(dataDEF['passes_medium']/dataDEF['passes'])/dataDEF['minutes']
dataDEF['pctpasseslong']=(dataDEF['passes_long']/dataDEF['passes'])/dataDEF['minutes']
dataDEF['pctpassescompleted']=dataDEF['passes_completed']/dataDEF['passes']/dataDEF['minutes']
#Creating a linear regression
trainDEF, testDEF = train_test_split(dataDEF, train_size=0.8)
modelDEF=smf.ols('ln(value)~age+CL+goals+xg_xa_per90+'
                  'passes_ground+touches_att_pen_area+touches_def_pen_area+aerials_won_pct'
                   '+'
                   ''
                   '+isPremierLeague+isLigue1'
                   '+Pts+xGA+xG',data=dataDEF)
modelDEF1=smf.ols('value~wins_gk+clean_sheets+Pts+W+GDiff+clean_sheets_pct+CL+xGDiff+GF+xG+passes_ground+passes_completed_medium+passes_medium+games+games_starts+minutes_90s+minutes+games_gk+games_starts_gk+minutes_90s_gk+minutes_gk+passes_throws_gk+passes_other_body+passes_completed+passes_received+passes_live+pass_targets+carries+touches_live_ball+passes_pct_long+touches_def_pen_area+passes_completed_short+passes_gk+passes_pressure+passes_pct+def_actions_outside_pen_area_gk+passes_total_distance+psxg_net_gk+touches_def_3rd+passes_short+passes+touches+ball_recoveries+through_balls+dribble_tackles_pct+psxg_net_per90_gk+passes_pct_launched_gk+save_pct+passes_low+xa_net+passes_progressive_distance+WinCL+carry_distance+gca_passes_dead+errors+passes_switches+passes_completed_long+crosses_gk+passes_intercepted+crosses_stopped_gk+dribbles_completed_pct+passes_left_foot+carry_progressive_distance+isPremierLeague+MP+avg_distance_def_actions_gk+saves+draws_gk+assists+goal_kicks+gca+foot_left+isLaLiga+passes_right_foot+shots_on_target_against+passes_pct_short+aerials_won_pct+passes_dead+assists_per90+gca_per90+passes_completed_launched_gk+passes_long+sca_passes_dead+def_actions_outside_pen_area_per90_gk+passes_pct_medium+crosses_stopped_pct_gk+passes_oob+own_goals_against_gk+gca_passes_live+pens_conceded+shots_on_target_pct+throw_ins+psxg_gk+pens_missed_gk+goals_assists_pens_per90+passes_received_pct+height+pens_allowed+goals_assists_per90+passes_launched_gk+npxg_net+pens_att_gk+cards_red+sca+xg_net+sca_passes_live+passes_high+fouled+free_kick_goals_against_gk+cards_yellow+corner_kicks_in+xa+passes_offsides+pens_saved+dribbles_completed+dribble_tackles+assisted_shots+players_dribbled_past+npxg_per_shot+xa_per90+passes_into_penalty_area+pressure_regain_pct+tackles_def_3rd+passes_free_kicks+miscontrols+dribbles+dribbles_vs+passes_head+isSerieA+clearances+corner_kick_goals_against_gk+dribbled_past+corner_kicks+shots_on_target_per90+tackles+goals_against_gk+pressures_def_3rd+tackles_won+dispossessed+tackles_mid_3rd+fouls+shots_total_per90+progressive_passes+offsides+npxg_xa_per90+xg_xa_per90+goals_pens_per90+passes_blocked+touches_mid_3rd+aerials_won+shots_on_target+sca_dribbles+gca_shots+pens_att+pens_made+pens_won+nutmegs+goals_per90+crosses+pressures+blocked_shots+pressure_regains+interceptions+goals_per_shot+shots_total+pressures_mid_3rd+shots_free_kicks+touches_att_pen_area+goals+sca_fouled+pressures_att_3rd+aerials_lost+touches_att_3rd+tackles_att_3rd+xg+goals_per_shot_on_target+own_goals+npxg+sca_shots+npxg_per90+xg_per90+blocks+blocked_passes+sca_per90+crosses_into_penalty_area+passes_into_final_third+D+psnpxg_per_shot_on_target_against+goal_kick_length_avg+foot_right+isBundesliga+isLigue1+passes_length_avg_gk+pct_goal_kicks_launched+losses_gk+pct_passes_launched_gk+age+goals_against_per90_gk+xGA+GA+L+LgRk+gca_dribbles+gca_fouled+gca_og_for+corner_kicks_out+corner_kicks_straight+foot_both+cards_yellow_red+blocked_shots_saves',data=dataDEF)    

#progressive_passesm
resultsDEF=modelDEF.fit()
resultsDEF_params=resultsDEF.params
resultsDEF1=modelDEF1.fit()
resultsDEF1_params=resultsDEF1.params
#Creating a robust regression
modelDEFrobust=sm.RLM(modelDEF.endog,modelDEF.exog,data=trainDEF).fit()
#r2_wls = sm.WLS(modelGK.endog, modelGK.exog, weights=modelGKrobust.weights).fit().rsquared
#modelGKrobustridge = sm.WLS(modelGK.endog, modelGK.exog, weights=modelGKrobust.weights).fit_regularized(alpha=0.2, L1_wt=0,start_params=resultsGK_params)
#print(modelGKrobust.summary(),
      #"R^2:", r2_wls)
finalDEF1 = sm.regression.linear_model.OLSResults(modelDEF, 
                                              modelDEFrobust.params, 
                                              modelDEF.normalized_cov_params)
#print(resultsDEF.get_robustcov_results(cov_type='HC1').summary())
finalDEF1.summary()
#ball recoveries, dribbled past, passes head - to describe
#print('do not bother about it printing summary like for OLS - this is a robust regression as you can see from the code')

In [ ]:
#VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor 
vif = pd.DataFrame()
vif["features"] = modelDEF.exog_names
vif["VIF Factor"] = [variance_inflation_factor(finalDEF1.model.exog, i) for i in range(finalDEF1.model.exog.shape[1])]
vif

In [ ]:
#testing
#Breusch-Pagan
bptestDEF=sm.stats.diagnostic.het_breuschpagan(finalDEF1.resid, finalDEF1.model.exog)
bptestDEF

In [ ]:
#Chowtest
trainDEF1=dataDEF[dataDEF['Season_201819#']==0]
trainDEF2=dataDEF[dataDEF['Season_201819#']==1]
JDEF = len(resultsDEF.params)-1 #number of coefficients
N1DEF = trainDEF1.shape[0]
N2DEF = trainDEF2.shape[0]
RSSdDEF=resultsDEF.ssr
resultsDEFridge1=smf.ols('ln(value)~age+CL+goals+xg_xa_per90+'
                  'passes_ground+touches_att_pen_area+touches_def_pen_area+aerials_won_pct'
                   '+'
                   ''
                   '+isPremierLeague+isLigue1'
                   '+Pts+xGA+xG',data=trainDEF1).fit()
RSSbDEF=resultsDEFridge1.ssr
kDEF = len(resultsDEFridge1.params)-1
resultsDEFridge2=smf.ols('ln(value)~age+CL+goals+xg_xa_per90+'
                  'passes_ground+touches_att_pen_area+touches_def_pen_area+aerials_won_pct'
                   '+'
                   ''
                   '+isPremierLeague+isLigue1'
                   '+Pts+xGA+xG',data=trainDEF2).fit()
RSSnbDEF = resultsDEFridge2.ssr
ChowDEF=((RSSdDEF-(RSSbDEF+RSSnbDEF))/JDEF)/((RSSbDEF+RSSnbDEF)/(N1DEF+N2DEF-2*kDEF))
pDEF=sp.stats.f.cdf(ChowDEF, JDEF, N1DEF+N2DEF-2*kDEF)
print(ChowDEF,pDEF,JDEF)

In [ ]:
predictionsDEF=finalDEF1.predict(dataDEF)
dataDEF['predsOLS']=np.exp(predictionsDEF)
#DEFpredtrain=resultsDEF.predict(trainDEF)
#DEFpredtest=resultsDEF.predict(testDEF)
#rmsetrainDEF=sm.tools.eval_measures.rmse(trainDEF['value'],np.exp(DEFpredtrain))
#rmsetestDEF=sm.tools.eval_measures.rmse(testDEF['value'],np.exp(DEFpredtest))
#print(rmsetrainDEF,rmsetestDEF)

In [ ]:
#cross validation
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
        #self.results_ = self.model_.fit_regularized(L1_wt=1, alpha=0.1,start_params=resultsDEF_params)
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

linearcval=cross_val_score(SMWrapper(sm.RLM), modelDEF.exog, modelDEF.endog, scoring='neg_root_mean_squared_error')
pd.DataFrame(pd.Series(np.transpose(linearcval)))
#robustcval=cross_val_score(SMWrapper(sm.RLM), modelDEF.exog, modelDEF.endog, scoring='neg_root_mean_squared_error')
#pd.DataFrame(pd.concat([pd.Series(np.transpose(linearcval)),pd.Series(np.transpose(robustcval))],axis=1))

In [ ]:
data=dataDEF[dataDEF['Season_201920#']==1]
#data1=data.sort_values('clean_sheetsm',ascending=False)[0:10]['value']
#data2=data.sort_values('psnpxg_per_shot_on_target_against',ascending=True)[0:10]['value']
#data3=data.sort_values('passes_pct_launched_gk',ascending=False)[0:10]['value']
data=data[['player','value','goals','xg_xa_per90','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct','passes_other_body']]
data1=data.sort_values('value',ascending=False)[0:10]
#p.DataFrame(data.mean()).transpose()
data1

In [ ]:
#applying millions formatter
def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.1fM' % (x * 1e-6)
formatter = mpl.ticker.FuncFormatter(millions)


#dataDEF=dataDEF[['goals','xg_xa_per90','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']

#dataDEF=dataDEF[dataDEF['goals']>0]
dataDEF=dataDEF[dataDEF['xg_xa_per90']>0]
dataDEF=dataDEF[dataDEF['passes_ground']>0]
dataDEF=dataDEF[dataDEF['touches_att_pen_area']>0]
dataDEF=dataDEF[dataDEF['touches_def_pen_area']>0]
dataDEF=dataDEF[dataDEF['aerials_won_pct']>0]
corrcoef1=np.corrcoef(dataDEF['value'],dataDEF['goals'])
corrcoef2=np.corrcoef(dataDEF['value'],dataDEF['xg_xa_per90'])
corrcoef3=np.corrcoef(dataDEF['value'],dataDEF['passes_ground'])
corrcoef4=np.corrcoef(dataDEF['value'],dataDEF['touches_att_pen_area'])
corrcoef5=np.corrcoef(dataDEF['value'],dataDEF['touches_def_pen_area'])
corrcoef6=np.corrcoef(dataDEF['value'],dataDEF['aerials_won_pct'])
corrcoef1=corrcoef1[0,1]
corrcoef2=corrcoef2[0,1]
corrcoef3=corrcoef3[0,1]
corrcoef4=corrcoef4[0,1]
corrcoef5=corrcoef5[0,1]
corrcoef6=corrcoef6[0,1]

fig, ax = plt.subplots(3, 2, figsize=(12, 12))
sns.regplot(ax=ax[0,0],x=dataDEF['goals'],y=dataDEF['value'],data=dataDEF,color='g')
sns.regplot(ax=ax[1,0],x=dataDEF['xg_xa_per90'],y=dataDEF['value'],data=dataDEF,color='blue')
sns.regplot(ax=ax[2,0],x=dataDEF['passes_ground'],y=dataDEF['value'],data=dataDEF,color='orange')
sns.regplot(ax=ax[0,1],x=dataDEF['touches_att_pen_area'],y=dataDEF['value'],data=dataDEF,color='cyan')
sns.regplot(ax=ax[1,1],x=dataDEF['touches_def_pen_area'],y=dataDEF['value'],data=dataDEF,color='magenta')
sns.regplot(ax=ax[2,1],x=dataDEF['aerials_won_pct'],y=dataDEF['value'],data=dataDEF,color='chocolate')

ax[0,0].yaxis.set_major_formatter(formatter)
ax[0,0].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[0,0].annotate("{:.2f}".format(corrcoef1),xy=(0.85,0.85), xycoords="axes fraction")

ax[1,0].yaxis.set_major_formatter(formatter)
ax[1,0].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[1,0].annotate("{:.2f}".format(corrcoef2),xy=(0.85,0.85), xycoords="axes fraction")

ax[2,0].yaxis.set_major_formatter(formatter)
ax[2,0].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[2,0].annotate("{:.2f}".format(corrcoef3),xy=(0.85,0.85), xycoords="axes fraction")

ax[0,1].yaxis.set_major_formatter(formatter)
ax[0,1].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[0,1].annotate("{:.2f}".format(corrcoef4),xy=(0.85,0.85), xycoords="axes fraction")

ax[1,1].yaxis.set_major_formatter(formatter)
ax[1,1].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[1,1].annotate("{:.2f}".format(corrcoef5),xy=(0.85,0.85), xycoords="axes fraction")

ax[2,1].yaxis.set_major_formatter(formatter)
ax[2,1].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[2,1].annotate("{:.2f}".format(corrcoef6),xy=(0.85,0.85), xycoords="axes fraction")